In [5]:
%load_ext autoreload
%autoreload 2

from CauseML.parameters import build_parameters_from_metric_levels
from CauseML.constants import Constants
from CauseML.data_generation import DataGeneratingProcessWrapper
import CauseML.data_sources as data_sources
from CauseML.utilities import extract_treat_and_control_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error as mse

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [8]:
def build_mlp():
    return MLPRegressor(
        hidden_layer_sizes=(50, 25, 10),
        activation='relu',
        solver='adam',
        batch_size='auto',
        learning_rate='constant',
        learning_rate_init=0.001,
        power_t=0.5,
        max_iter=1000,
        shuffle=True,
        random_state=1,
        tol=0.0001,
        verbose=False,
        warm_start=False,
        early_stopping=True,
        validation_fraction=0.10,
        epsilon=1e-08,
        n_iter_no_change=5)

In [9]:
PARTIAL_CORRELATION_DEG = 0.1
def generate_synthetic_data(
    prop_score=0.5,
    n_covars=20,
    n_observations=50000):
    
    covar_data_source = data_sources.load_random_normal_covariates(
        n_covars=n_covars,
        n_observations=n_observations,
        partial_correlation_degree=PARTIAL_CORRELATION_DEG)
    covar_data = covar_data_source.original_covariate_data.to_numpy()
    outcome_coeffs = np.random.uniform(-5, 5, size=20)
    
    noise = np.random.normal(size=n_observations)
    Y0 = (covar_data @ outcome_coeffs) + \
        5*((covar_data[:, 0] > 0.5).astype(int)) + \
        noise
    Y1 = Y0 + 8*((covar_data[:, 1] > 0.1).astype(int))
    
    TE = Y1 - Y0
    T = (np.random.uniform(0, 1, size=n_observations) < prop_score).astype(int)
    Y = T*Y1 + (1-T)*Y0
    
    obs_data = covar_data_source.original_covariate_data.copy()
    obs_data["T"] = T
    obs_data["Y"] = Y
    
    oracle_data = pd.DataFrame({
        "TE": TE,
        "NOISE(Y)": noise,
        "Y0": Y0,
        "Y1": Y1,
        "P(T|X)": prop_score
    })
    
#     obs_data = dgp_wrapper.get_observed_data()
#     oracle_data = dgp_wrapper.get_oracle_data()
    
    return obs_data, oracle_data

In [10]:
def generate_hybrid_data(
    prop_score=0.5,
    n_covars=20,
    n_observations=50000):
    
    covar_data_source = data_sources.load_random_normal_covariates(
        n_covars=n_covars,
        n_observations=n_observations,
        partial_correlation_degree=PARTIAL_CORRELATION_DEG)

    dgp_params = build_parameters_from_metric_levels({
        Constants.MetricNames.TREATMENT_NONLINEARITY: Constants.MetricLevels.LOW,
        Constants.MetricNames.OUTCOME_NONLINEARITY: Constants.MetricLevels.LOW,
        Constants.MetricNames.TE_HETEROGENEITY: Constants.MetricLevels.LOW
    })
    
    dgp_params.set_parameter("MIN_PROPENSITY_SCORE", 0.001)
    dgp_params.set_parameter("MAX_PROPENSITY_SCORE", 0.9999)
    dgp_params.set_parameter("TREATMENT_EFFECT_HETEROGENEITY", 0.05)
    dgp_params.set_parameter("OUTCOME_NOISE_TAIL_THICKNESS", 300)
    dgp_params.set_parameter("TARGET_PROPENSITY_SCORE", prop_score)
    dgp_params.set_parameter("TREAT_MECHANISM_COVARIATE_SELECTION_PROBABILITY",
                             {
                                "LINEAR": 1,
                                "POLY_QUAD": 0.0,
                                "POLY_CUBIC": 0.0,
                                "STEP_JUMP": 0,
                                "STEP_KINK": 0,
                                "INTERACTION_TWO_WAY": 0,
                                "INTERACTION_THREE_WAY": 0
                             })
    dgp_params.set_parameter("OUTCOME_MECHANISM_COVARIATE_SELECTION_PROBABILITY",
                             {
                                "LINEAR": 1,
                                "POLY_QUAD": 0.0,
                                "POLY_CUBIC": 0.0,
                                "STEP_JUMP": 0.05,
                                "STEP_KINK": 0,
                                "INTERACTION_TWO_WAY": 0,
                                "INTERACTION_THREE_WAY": 0
                             })

    dgp_wrapper = DataGeneratingProcessWrapper(
        parameters=dgp_params, data_source=covar_data_source)

    dgp_wrapper.sample_dgp()

    _ = dgp_wrapper.generate_data()
    
    obs_data = dgp_wrapper.get_observed_data()
    oracle_data = dgp_wrapper.get_oracle_data()
    
    return obs_data, oracle_data

In [55]:
observed_data, unobservable_data = generate_hybrid_data(prop_score=0.01)
observed_data, unobservable_data = generate_synthetic_data(prop_score=0.01)
observed_data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X12,X13,X14,X15,X16,X17,X18,X19,T,Y
0,-0.182374,0.004818,0.063120,-0.053970,-0.417480,0.130778,0.263482,-0.030452,-0.098303,0.230740,...,0.276854,0.106865,-0.152783,-0.340035,-0.148173,0.241656,-0.207254,0.122505,0,-3.318321
1,-0.449061,-0.615149,0.149645,0.123435,-0.005158,-0.365969,0.197851,-0.325774,-0.263456,-0.230029,...,-0.601336,-0.453600,-0.214771,0.105800,0.485089,0.284280,0.468553,-0.700081,0,-4.239334
2,-0.369948,-0.403287,-0.051238,0.176380,-0.096799,-0.828878,0.453145,-0.398782,0.023587,0.369469,...,-0.507002,0.054135,-0.541419,0.003777,0.333014,0.150174,0.524723,-0.551230,0,-2.395443
3,-0.103530,-0.206862,0.224169,-0.116494,-0.268307,-0.283045,0.367962,-0.008625,0.407226,-0.097554,...,0.216358,-0.365650,-0.325156,-0.038668,0.237870,0.465787,-0.036785,-0.502184,0,-0.905016
4,-0.473127,0.308793,0.209059,-0.200980,0.105340,0.130716,-0.000915,-0.073977,-0.525490,-0.004810,...,-0.511083,-0.151388,0.788584,-0.131746,-0.113652,-0.047416,0.141409,-0.197000,0,-1.751199


In [56]:
unobservable_data.head()

,TE,NOISE(Y),Y0,Y1,P(T|X)
0,0.0,-1.251275,-3.318321,-3.318321,0.01
1,0.0,0.083520,-4.239334,-4.239334,0.01
2,0.0,-1.268570,-2.395443,-2.395443,0.01
3,0.0,0.005135,-0.905016,-0.905016,0.01
4,8.0,-0.759979,-1.751199,6.248801,0.01


In [11]:
def score_ITE(ITE_true, ITE_pred):
    return mse(ITE_true, ITE_pred)

In [12]:
def T_learner(data):
    treated_data, control_data = extract_treat_and_control_data(
        data, data["T"])
    
    # train u1
    u1 = build_mlp()
    X_treated = treated_data.drop(["T", "Y"], axis=1)
    u1.fit(X_treated, treated_data["Y"])
    
    # train u0
    u0 = build_mlp()
    X_control = control_data.drop(["T", "Y"], axis=1)
    u0.fit(X_control, control_data["Y"])
    
    # Generate potential outcome predictions
    X = data.drop(["T", "Y"], axis=1)
    y_1_predicted = u1.predict(X)
    y_0_predicted = u0.predict(X)
    
    ITE_pred = y_1_predicted - y_0_predicted
    
    return ITE_pred, u1, u0

In [13]:
def S_learner(data):
    # train u1
    u = build_mlp()
    X = data.drop("Y", axis=1)
    u.fit(X, data["Y"])
    
    # Generate potential outcomes
    X_under_treatment = X.copy()
    X_under_treatment["T"] = 1
    
    X_under_control = X.copy()
    X_under_control["T"] = 0
    
    y_1_predicted = u.predict(X_under_treatment)
    y_0_predicted = u.predict(X_under_control)
    y_1_predicted - y_0_predicted
    
    ITE_pred = y_1_predicted - y_0_predicted
    
    return  ITE_pred

In [14]:
def X_learner(data, prop_score, u1=None, u0=None):
    treated_data, control_data = extract_treat_and_control_data(
        data, data["T"])
    X_treated = treated_data.drop(["T", "Y"], axis=1)
    X_control = control_data.drop(["T", "Y"], axis=1)

    if u1 is None:
        # train u1
        u1 = build_mlp()
        u1.fit(X_treated, treated_data["Y"])
    
    if u0 is None:
        # train u0
        u0 = build_mlp()
        u0.fit(X_control, control_data["Y"])
    
    # Find individual treatment effects for treated/controls
    itet = treated_data["Y"] - u0.predict(X_treated)
    itec =  u1.predict(X_control) - control_data["Y"]
    
    # Fit models to the treated and controls
    te1 = build_mlp()
    te1.fit(X_treated, itet)
    
    te0 = build_mlp()
    te0.fit(X_control, itec)
    
    X = data.drop(["T", "Y"], axis=1)
    te1_predicted = te1.predict(X)
    te0_predicted = te0.predict(X)
    
    # Estimate prop scores
#     lm = LogisticRegression(solver="lbfgs")
#     lm.fit(X, data["T"])
#     prop_scores = lm.predict(X)
    
    # Down-weight observes which are likely to belong
    # to other class.
    te = prop_score*te0_predicted + (1-prop_score)*te1_predicted
    
    return te

In [66]:
ITE_pred, u1, u0 = T_learner(observed_data)
score_ITE(unobservable_data["TE"], ITE_pred)

6.304035808777853

In [67]:
ITE_pred = S_learner(observed_data)
score_ITE(unobservable_data["TE"], ITE_pred)

2.6395543639347854

In [68]:
ITE_pred = X_learner(observed_data, 0.01, u1, u0)
score_ITE(unobservable_data["TE"], ITE_pred)

5.805592018489937

In [15]:
def run_trial(prop_score=0.5, n_covars=20, n_observations=50000):
    observed_data, unobservable_data = generate_synthetic_data(
        prop_score=prop_score,
        n_covars=n_covars,
        n_observations=n_observations)
    
    ITE_true = unobservable_data["TE"]
    
    # T learner
    ITE_pred, u1, u0 = T_learner(observed_data)
    T_loss = score_ITE(ITE_true, ITE_pred)
    
    # S learner
    ITE_pred = S_learner(observed_data)
    S_loss = score_ITE(ITE_true, ITE_pred)
    
    # X learner
    ITE_pred = X_learner(observed_data, prop_score, u1, u0)
    X_loss = score_ITE(ITE_true, ITE_pred)
    
    return T_loss, S_loss, X_loss

In [16]:
N_obs = np.logspace(3.75, 5.5, 7).astype(int)
# N_obs = [100000, 200000, 300000]
N_obs

array([  5623,  11006,  21544,  42169,  82540, 161559, 316227])

In [ ]:
%%time

results = []
N_trials = 20
for n_obs in N_obs:
    print("\nRunning at n:", n_obs, end=": ")
    trial_results = []
    for i in range(N_trials):
        print(i, end=",")
        res = run_trial(prop_score=0.02, n_observations=n_obs)
        trial_results.append(res)
        
    
    trial_results = np.array(trial_results)
    results.append(np.mean(trial_results, axis=0))


Running at n: 5623: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
Running at n: 11006: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
Running at n: 21544: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
Running at n: 42169: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
Running at n: 82540: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
Running at n: 161559: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
Running at n: 316227: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,

In [ ]:
results = np.array(results)
names = ["T learner", "S learner", "X learner"]
for i, name in enumerate(names):
    plt.scatter(np.log(N_obs), np.log(results[:, i]), label=name)

plt.legend()
plt.show()